# Deploy to Vespa Cloud

![Vespa Cloud logo](https://cloud.vespa.ai/assets/logos/vespa-cloud-logo-full-black.png)

This notebook deploys a Vespa.ai sample application to Vespa Cloud using pyvespa.
The [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html) is used for managing security credentials.

See pyvespa, and jupyter notebooks [requirements](https://pyvespa.readthedocs.io/en/latest/index.html#requirements)
to run this notebook.

Refer to the [sample apps](https://github.com/vespa-engine/sample-apps) site for more applications.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vespa-engine/pyvespa/blob/master/docs/sphinx/source/deploy-vespa-cloud.ipynb)

## Install pyvespa and the Vespa CLI

In [ ]:
!pip install pyvespa

If homebrew is installed, install the Vespa CLI:

In [ ]:
!brew install vespa-cli

Alternatively, when running in Colab, install the Vespa CLI:

In [ ]:
import os
import requests
res = requests.get(url="https://api.github.com/repos/vespa-engine/vespa/releases/latest").json()
os.environ["VERSION"] = res["tag_name"].replace("v", "")
!curl -fsSL https://github.com/vespa-engine/vespa/releases/download/v${VERSION}/vespa-cli_${VERSION}_linux_amd64.tar.gz | tar -zxf -
!ln -sf /content/vespa-cli_${VERSION}_linux_amd64/bin/vespa /usr/local/bin/vespa

## Create a Vespa Cloud tenant

Sign up and create a tenant: Run steps 1 and 2 in [getting started](https://cloud.vespa.ai/en/getting-started), then replace the tenant name below:

In [5]:
import os
os.environ["TENANT_NAME"] = "tenant_name_here"

## Set up security credentials

In [ ]:
!vespa config set target cloud
!vespa config set application ${TENANT_NAME}.myapp.default
!vespa auth cert -N
!vespa auth api-key

Set API key location. The `vespa` command stores the credentials in ~/.vespa, like:

      /Users/me/.vespa/mytenant.api-key.pem
      /Users/me/.vespa/mytenant.myapp.default/data-plane-public-cert.pem
      /Users/me/.vespa/mytenant.myapp.default/data-plane-private-key.pem

Enter full path to the api key (like `/Users/me/.vespa/mytenant.api-key.pem`):

In [7]:
api_key_path = "path_here"

## Create an application package

An [application package](https://pyvespa.readthedocs.io/en/latest/use_cases/text_search/text-search-quick-start.html)
is the application's configuration, like schema.
Here, using a basic question answering app from the app gallery.

In [8]:
from vespa.gallery import QuestionAnswering

app = QuestionAnswering()

## Create a VespaCloud instance

Refer to the [VespaCloud reference](reference-api.rst#vespacloud).

In [9]:
from vespa.deployment import VespaCloud

vespa_cloud = VespaCloud(
    tenant=os.getenv("TENANT_NAME"),
    application="myapp",
    key_location=api_key_path,
    application_package=app)

## Deploy to Vespa Cloud

Refer to [tenants, applications and instances](https://cloud.vespa.ai/en/tenant-apps-instances) for details - here creating an instance named `default`:

In [ ]:
app = vespa_cloud.deploy(instance="default")

That is it, you can now interact with your deployed application through the `app` instance.